In [1]:
from ase.io import read
import time
import sys
sys.path.append('../code')
from Generate_Descriptors import get_dscribe_descriptors

In [33]:
C7O2H10_structs = read("../data/C7O2H10.extxyz.gz", ":2000")

In [3]:
graphenes = read("../data/ManyGraphenes.extxyz.gz")

In [12]:
def TickTock(func, *args, **kwargs):
    tick = time.time()
    func_output = func(*args, **kwargs)
    tock = time.time()
    return func_output, tock - tick

In [34]:
species = ["C", "O", "H", "N", "Ca", "Li"]
rcut = 3
nmax = 12
lmax = 9
SoapList, SoapTime = TickTock(get_dscribe_descriptors, C7O2H10_structs, species = species, rcut = rcut, nmax = nmax, lmax = lmax, is_global=False, return_derivatives = False)
print(SoapTime)

23.963632822036743


In [41]:
species = ["C", "O", "H", "N", "Ca", "Li"]
rcut = 3
nmax = 12
lmax = 9
SoapList, SoapTime = TickTock(get_dscribe_descriptors, graphenes, species = species, rcut = rcut, nmax = nmax, lmax = lmax, is_global=False, return_derivatives = False)
print(SoapTime)

0.09846997261047363


In [44]:
from ase.build import bulk
diamond = bulk("C", 'diamond')
diamond
Structure3DPlot(diamond).Plot()

Atoms(symbols='C2', pbc=True, cell=[[0.0, 1.785, 1.785], [1.785, 0.0, 1.785], [1.785, 1.785, 0.0]])

In [ ]:
# Breakdown of the derivatives output tensor from soap.derivatives(). This explains what the different indices mean in different situations

use_atom_centers = True
use_local_descriptors = True
n_structures = 5

averaging_keyword = "off" if use_local_descriptors else "outer"
my_water_molecule = ase.build.molecule('H2O')
if n_structures == 1:
    my_atoms_list = my_water_molecule
    print("This breakdown of the derivatives tensor is for a single (1) water molecule.")
elif n_structures > 1:
    my_atoms_list = [my_water_molecule] * n_structures
    print("This breakdown of the derivatives tensor is for {} water molecules.".format(n_structures))
    print("To access the tensor values without defining a tensor for each molecule, add an extra index in front. I didn't do this here because it was easier not to.")


soap = SOAP(average=averaging_keyword,  species=["H", "O"], periodic=False, rcut=1.2, nmax=1, lmax=1)

if use_atom_centers:
    positions = my_water_molecule.get_positions()
else:
    positions = [[0, 0, 0],[5,2,3]]

if n_structures == 1:
    positions_list = positions
elif n_structures > 1:
    positions_list = [positions]*len(my_atoms_list)

my_soap = soap.create(my_atoms_list)
derivatives, descriptors = soap.derivatives(my_atoms_list, positions=positions_list , method="auto")

print("")

for k in range(n_structures):
    if n_structures == 1:
        derivative_k = derivatives
        print("For the water molecule soap.derivatives() output, D (soap derivatives) and S (soaps):")
    elif n_structures > 1:
        derivative_k = derivatives[k]
        print("For water molecule #{} soap.derivatives() output, D (soap derivatives) and S (soaps):".format(k))
    for i in range(len(derivative_k)):
        print("---------------------------------------------------------------------------------------------")
        print("---------------------------------------------------------------------------------------------")
        if use_local_descriptors:
            atom_i = "atom " + my_water_molecule[i].symbol + str(my_water_molecule[i].index) + " " if use_atom_centers else ""
            print("D[{}] = derivatives of S[{}], a local soap descriptor centered at {}({:5.2f}, {:5.2f}, {:5.2f})".format(i, i, atom_i, *positions[i]))
        else:
            print("D[{}] = derivative of S[{}], the global soap descriptor averaged over {} centers".format(i, i, len(positions[i])))
        print("---------------------------------------------------------------------------------------------")
        print("---------------------------------------------------------------------------------------------")
        for j in range(len(my_water_molecule)):
            atom_j = my_water_molecule[j].symbol + str(my_water_molecule[j].index)
            print("D[{}, {}] = derivative of S[{}] with respect to atom {} positions".format(i, j, i, atom_j))
            print("--------------------------------------------------------------")

            print("D[{}, {}, {}] = dS[{}]/dX{}: ".format(i, j, 0, i, j), "[" + "  ".join(["{:7.4f}".format(p) for p in derivative_k[i,j,0]]) + "]")
            print("D[{}, {}, {}] = dS[{}]/dY{}: ".format(i, j, 1, i,j), "[" + "  ".join(["{:7.4f}".format(p) for p in derivative_k[i,j,1]]) + "]")
            print("D[{}, {}, {}] = dS[{}]/dZ{}: ".format(i, j, 2, i,j), "[" + "  ".join(["{:7.4f}".format(p) for p in derivative_k[i,j,2]]) + "]")
            #print("")
            print("--------------------------------------------------------------\n")

